## Model Training Preparation (PySpark)

In this step, we initialize a local Spark session and prepare the final dataset
for machine learning. Feature columns are assembled into a single feature vector,
and the dataset is split into training and test sets for model evaluation.


In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pandas as pd

spark = SparkSession.builder \
    .appName("ALAZ_Training_Phase") \
    .master("local[3]") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

df = spark.read.parquet("../data/processed/ALAZ_Final_Dataset_Cleaned.parquet")

feature_cols = [
    'yukselti', 'egim', 'baki',
    'sicaklik', 'ruzgar_hizi', 'bagil_nem', 'yagis', 'toprak_sicakligi',
    'NDVI'
]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_ml = assembler.transform(df)

final_data = df_ml.select("features", "label")
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

print(f"Eğitim Verisi: {train_data.count()}")
print(f"Test Verisi: {test_data.count()}")

Eğitim Verisi: 296136


[Stage 5:=======================================>                   (2 + 1) / 3]

Test Verisi: 73964


In [ ]:
print("Random Forest Modeli Eğitiliyor...")

rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=50, maxDepth=10, seed=42)

model = rf.fit(train_data)

print("Model Eğitimi Tamamlandı")

print("Test verisi üzerinde tahmin:")
predictions = model.transform(test_data)

predictions.select("label", "prediction", "probability").show(10, truncate=False)

## Model Performance Evaluation

In this step, we evaluate the performance of the trained Random Forest model using the following metrics:

- **Accuracy**: The overall correctness of the model.
- **F1 Score**: The harmonic mean of precision and recall.
- **Weighted Precision**: The precision of the model, considering class imbalances.
- **Weighted Recall**: The recall of the model, considering class imbalances.

The evaluation results are displayed as percentages for each metric.


In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
f1_score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
weighted_precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
weighted_recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

print(f"MODEL BAŞARI RAPORU")
print(f"Accuracy)          : %{accuracy*100:.2f}")
print(f"F1 Skoru           : %{f1_score*100:.2f}")
print(f"Precision(Kesinlik): %{weighted_precision*100:.2f}")
print(f"Recall(Duyarlılık) : %{weighted_recall*100:.2f}")

## Model Training with 100 Trees

In this step, we train a new Random Forest model with the following parameters:

- **Number of Trees**: 100
- **Maximum Depth**: 15

The model is evaluated using the same metrics as before: **Accuracy** and **F1 Score**.


In [ ]:
print("Random Forest 100 Ağaç...")

rf_v2 = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100, maxDepth=15, seed=42)

model_v2 = rf_v2.fit(train_data)

predictions_v2 = model_v2.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
acc_v2 = evaluator.evaluate(predictions_v2, {evaluator.metricName: "accuracy"})
f1_v2 = evaluator.evaluate(predictions_v2, {evaluator.metricName: "f1"})


print(f"YENİ SKORLAR")
print(f"Accuracy: %{acc_v2*100:.2f}")
print(f"F1 Skoru: %{f1_v2*100:.2f}")

## Feature Importance Visualization

In this step, we visualize the importance of each feature in predicting wildfire risk using the trained Random Forest model. The features are sorted by their importance, and a bar plot is generated to show the relative importance of each feature.

The most influential factors are identified to understand which variables are driving the model's predictions.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

feature_list = [
    'yukselti', 'egim', 'baki',
    'sicaklik', 'ruzgar_hizi', 'bagil_nem', 'yagis', 'toprak_sicakligi',
    'NDVI'
]

importances = model_v2.featureImportances

fi_df = pd.DataFrame(list(zip(feature_list, importances)), columns=["Feature", "Importance"])
fi_df = fi_df.sort_values(by="Importance", ascending=False)

print(fi_df)

plt.figure(figsize=(10, 6))
sns.barplot(x="Importance", y="Feature", data=fi_df, palette="viridis")
plt.title("Yangın Riskini Etkileyen En Önemli Faktörler")
plt.xlabel("Önem Derecesi")
plt.ylabel("Faktörler")
plt.show()

## Confusion Matrix

This section presents the confusion matrix of the trained Random Forest model.
It provides a detailed view of the model’s classification performance by showing
the distribution of true positives, true negatives, false positives, and false negatives.

The heatmap visualization makes it easier to interpret how well the model distinguishes
between wildfire and non-wildfire classes.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

conf_matrix_spark = predictions_v2.groupBy("label", "prediction").count()
conf_matrix_pd = conf_matrix_spark.toPandas()

heatmap_data = conf_matrix_pd.pivot(index='label', columns='prediction', values='count').fillna(0)

heatmap_data = heatmap_data.sort_index(ascending=True).sort_index(axis=1, ascending=True)

heatmap_data.index = ["Gerçek: Yok (0)", "Gerçek: Var (1)"]
heatmap_data.columns = ["Tahmin: Yok (0)", "Tahmin: Var (1)"]

plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)

sns.heatmap(heatmap_data, annot=True, fmt='g', cmap='Blues', cbar=False, linewidths=1, linecolor='black')

plt.title('ALAZ Model Performansı: Karışıklık Matrisi', fontsize=16)
plt.yticks(rotation=0)
plt.show()

## Wildfire Risk Mapping: Manavgat Case Study

This section applies the trained Random Forest model to the Manavgat region
for July 28, 2021, one of the most severe wildfire events in Türkiye.

The model generates a spatial wildfire risk map by predicting fire probabilities
for each grid point. Observed wildfire locations are overlaid on the map to
visually compare predicted risk levels with actual fire occurrences.


In [ ]:
from pyspark.sql.functions import col

print("🗺️ Manavgat Yangını (28 Temmuz 2021) verileri çekiliyor...")

df_manavgat = df.filter(
    (col("tarih") == "2021-07-28") &
    (col("latitude").between(36.5, 37.5)) &
    (col("longitude").between(31.0, 32.5))
)

count = df_manavgat.count()

if count > 0:
    df_manavgat_ml = assembler.transform(df_manavgat)

    predictions_manavgat = model_v2.transform(df_manavgat_ml)

    result_pd = predictions_manavgat.select("latitude", "longitude", "probability", "label").toPandas()

    import matplotlib.pyplot as plt

    result_pd["risk_score"] = result_pd["probability"].apply(lambda x: x[1])

    plt.figure(figsize=(12, 8))
    
    sc = plt.scatter(
        result_pd["longitude"], 
        result_pd["latitude"], 
        c=result_pd["risk_score"], 
        cmap="RdYlGn_r",
        s=20,
        alpha=0.8
    )

    plt.colorbar(sc, label="Yangın Risk Skoru (0-1)")
    plt.title("ALAZ Risk Haritası: Manavgat Bölgesi (28 Temmuz 2021)", fontsize=15)
    plt.xlabel("Boylam")
    plt.ylabel("Enlem")
    plt.grid(True, linestyle="--", alpha=0.5)
    real_fires = result_pd[result_pd["label"] == 1]
    plt.scatter(real_fires["longitude"], real_fires["latitude"], c='black', marker='x', label='Gerçekleşen Yangınlar', s=50)
    plt.legend(loc='upper right')

    plt.show()
else:
    print("Tarihi veya koordinatları kontrol et.")